In [10]:
cd

/suphys/aloe8475


In [11]:
cd "Documents/edamame"

/import/silo2/aloe8475/Documents/edamame


In [13]:
from scipy.io import loadmat, savemat
from scipy.stats import kurtosis, skew

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime
import networkx as nx
from edamame import *
from tqdm import tqdm_notebook
import os
import edamame.core.wires as wires
from random import choice
from IPython.core.debugger import set_trace
from collections import Counter
from scipy import sparse
from scipy.stats import entropy

In [27]:
#Python 3:
import pickle 
import _pickle as cPickle
# import cPickle
import gzip
def compressed_pickle(obj, filename,protocol=-1):
    with gzip.open(filename, 'wb') as f:
        cPickle.dump(obj, f, protocol)

In [28]:
def decompress_pickle(file):
    with gzip.open(file, 'rb') as f:
        loaded_object = cPickle.load(f)
        return loaded_object

In [29]:
cd "/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks/"

/import/silo2/aloe8475/Documents/CODE/Analysis/Functional Connectivity/Functional Tasks


In [30]:
name='networks_LinearTransformation.pkl'
# print "Loading Networks"
file = open(name, 'rb')
#     [ASN300,cluster1,cluster2,cluster3,time_index,nodesList] = pickle.load(file)
#     [ASN300,cluster1,cluster2,cluster3] = pickle.load(file)
[ASN300] = pickle.load(file)

In [112]:
g=[]
for i in range(len(ASN300)):
    for j in range(len(ASN300[i])):
        g.append(ASN300[i][j]['G'])

In [31]:
"""
schieber.py
-----------

Python implementation of the distance method in 'Quantification of network
structural dissimilarities', by Schieber et al.

"""

def jensen_shannon(dists):
    """Jensen-Shannong entropy of a family of distributions.

    dists is a N by M matrix where each row is the ditribution over a set
    of M elements.

    """
    size = dists.shape[0]
    vec = np.log(dists.sum(axis=0)) - np.log(size)
    first_term = (-1/size) * dists.dot(vec).sum()
    second_term = entropy(dists.T).mean()
    return first_term - second_term


def nnd(graph, dists=None):
    """Compute Network Node Dispersion (NND)."""
    if dists is None:
        dists = node_distance(graph)
    diam = dists.shape[1]
    return jensen_shannon(dists) / np.log(diam + 1)


def node_distance(graph):
    """All shortest path distances.

    The nodes must be labeled by integers from 0 to graph.order() - 1.

    """
    size = graph.order()
    if size < 2:
        return 1

    result = np.zeros((size, size)) # does this need to be sparse?
    dists = dict(nx.shortest_path_length(graph))  
    dists = np.array([[dists[n1][n2] if dists[n1][n2] < np.inf else size
                       for n2 in dists[n1]]
                      for n1 in dists])

    for idx, row in enumerate(dists):
        counts = Counter(row)
        result[idx] = [counts[l] for l in range(size)]

    diam = (result.sum(axis=0) > 0).sum()
    result = result[:, :diam]

    return result / size


def alpha_centrality(graph, normalize=False):
    """Bonacich centrality."""
    size = graph.order()
    degrees = graph.degree()
    degrees = np.array([degrees[n] for n in graph.nodes()]) / (size - 1)
    alpha = 1 / size
    exogenous = degrees
    mat = sparse.identity(size) - alpha * nx.adjacency_matrix(graph).T
    res = sparse.linalg.inv(mat.asformat('csc')).dot(exogenous)
    return res if not normalize else res / res.sum()


def pad(array, num_cols):
    """Pad with all-zero columns."""
    rows = array.shape[0]
    cols_to_add = num_cols - array.shape[1]
    return np.hstack([array, np.zeros((rows, cols_to_add))])


def schieber(graph1, graph2, w1, w2, w3=None, complement=False):
    """Distance between two graphs. See eqn 2 in the paper."""
    dists1 = node_distance(graph1)
    dists2 = node_distance(graph2)
    if dists1.shape[1] > dists2.shape[1]:
        pad(dists2, dists1.shape[1])
    elif dists2.shape[1] > dists1.shape[1]:
        pad(dists1, dists2.shape[1])

    first_term = np.vstack([dists1.mean(axis=0), dists2.mean(axis=0)])
    first_term = w1 * np.sqrt(jensen_shannon(first_term) / np.log(2))

    second_term = np.sqrt(nnd(graph1, dists1)) - np.sqrt(nnd(graph2, dists2))
    second_term = w2 * np.abs(second_term)

    if w3 is not None:
        alpha1 = alpha_centrality(graph1, normalize=True)
        alpha2 = alpha_centrality(graph2, normalize=True)
        all_alphas = np.vstack([alpha1, alpha2])
        third_term = np.sqrt(jensen_shannon(all_alphas) / np.log(2))
        if complement:
            alpha_comp1 = alpha_centrality(nx.complement(graph1), normalize=True)
            alpha_comp2 = alpha_centrality(nx.complement(graph2), normalize=True)
            all_alphas = np.vstack([alpha_comp1, alpha_comp2])
            third_term += np.sqrt(jensen_shannon(all_alphas) / np.log(2))
        third_term = w3 * third_term / 2
        return first_term + second_term + third_term
    else:
        return first_term + second_term

In [36]:
#resize graphs:
def pad_graphs(arrs): #find differences 
    lens = [len(i) for i in arrs]
    arr = np.ma.empty((np.max(lens),len(arrs)))
    arr.mask = True
    for idx, l in enumerate(arrs):
        arr[:len(l),idx] = l
    return(arr)

In [128]:
nx.adjacency_matrix(g[i]).todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [132]:
np.pad(nx.adjacency_matrix(g[i]).todense(),(0,1),'constant', constant_values=np.nan)

array([[ 0.,  0.,  0., ...,  0.,  0., nan],
       [ 0.,  0.,  1., ...,  0.,  0., nan],
       [ 0.,  1.,  0., ...,  0.,  0., nan],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0., nan],
       [ 0.,  0.,  0., ...,  0.,  0., nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [133]:
graphs=[]
for i in range(len(g)):
    if len(g[i])<300:
        graphs.append((np.pad(nx.adjacency_matrix(g[i]).todense(),(0,1),'constant', constant_values=np.nan)))

In [135]:
"""Compute distance between pre-computed graphs."""
D=[]
for i in range(len(g)):
    for j in range(len(g)):
        D.append(schieber(g[i],g[j],0.45, 0.45, 0.1,True))

# graph = nx.barabasi_albert_graph(200,5)
# print(schieber(graph, graph, 0.5, 0.5))
# print(schieber(graph, graph, 0.45, 0.45, 0.1))
# print(schieber(graph, graph, 0.45, 0.45, 0.1, True))



ValueError: all the input array dimensions except for the concatenation axis must match exactly